# Day 09. Exercise 02
# Metrics

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
df_dayofweek = pd.read_csv('../data/dayofweek.csv', usecols=['dayofweek'])

In [3]:
df['dayofweek'] = df_dayofweek
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [4]:
X = df.drop(columns=['dayofweek'])
y = df['dayofweek']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [6]:
svc = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', random_state=21, probability=True)
svc.fit(X_train, y_train)

SVC(C=10, gamma='auto', probability=True, random_state=21)

In [7]:
y_pred_svc = svc.predict(X_test)

In [8]:
accuracy_score(y_test, y_pred_svc)

0.8875739644970414

In [9]:
precision_score(y_test, y_pred_svc, average='weighted')

0.8926729169690374

In [10]:
recall_score(y_test, y_pred_svc, average='weighted')

0.8875739644970414

In [11]:
roc_auc_score(y_test, svc.predict_proba(X_test), multi_class='ovr')

0.9775092370859483

## 3. Decision tree

1. The same task for decision tree

In [12]:
tree = DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=22, random_state=21)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=22, random_state=21)

In [13]:
y_pred_tree = tree.predict(X_test)

In [14]:
print(accuracy_score(y_test, y_pred_tree))
print(precision_score(y_test, y_pred_tree, average='weighted'))
print(recall_score(y_test, y_pred_tree, average='weighted'))
print(roc_auc_score(y_test, svc.predict_proba(X_test), multi_class='ovr', average='weighted'))

0.8905325443786982
0.8926192681313897
0.8905325443786982
0.9816828267059918


## 4. Random forest

1. The same task for random forest.

In [15]:
rf = RandomForestClassifier(class_weight=None, criterion='gini', max_depth=28, n_estimators=50, random_state=21)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=28, n_estimators=50, random_state=21)

In [16]:
y_pred_rf = rf.predict(X_test)

In [17]:
print(accuracy_score(y_test, y_pred_rf))
print(precision_score(y_test, y_pred_rf, average='weighted'))
print(recall_score(y_test, y_pred_rf, average='weighted'))
print(roc_auc_score(y_test, svc.predict_proba(X_test), multi_class='ovr', average='weighted'))

0.9289940828402367
0.9300865038851309
0.9289940828402367
0.9816828267059918


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [18]:
best_model = RandomForestClassifier(class_weight=None, criterion='gini', max_depth=28, n_estimators=50, random_state=21)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
joblib.dump(best_model, '../data/model_ex_02.pkl')

['../data/model_ex_02.pkl']

In [19]:
cm = confusion_matrix(y_test, y_pred)

In [20]:
class_sum = cm.sum(axis=1)
errors_class = class_sum - np.diag(cm)
error_rates = (errors_class / class_sum) * 100
for i, errors in enumerate(error_rates):
    print(f'Class {i}: count errors = {errors:.1f}%')

Class 0: count errors = 25.9%
Class 1: count errors = 10.9%
Class 2: count errors = 6.7%
Class 3: count errors = 2.5%
Class 4: count errors = 14.3%
Class 5: count errors = 5.6%
Class 6: count errors = 1.4%


In [21]:
def errors_feature(feature):
    col_feature = X_test.columns.str.contains(feature)
    df_feature = X_test[X_test.columns[col_feature]]
    no_pred = y_pred != y_test
    feature = df_feature[no_pred].sum().sort_values(ascending=False)
    return feature.index[0], feature.iloc[0]

In [22]:
errors_feature('labname'), errors_feature('uid')

(('labname_project1', 9.0), ('uid_user_19', 4.0))

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [23]:
def metrics_models(models, params):
    if len(models) != len(params):
        print('The number of models must match the number of parameters')
    else:
        for number in range(len(models)):
            model = models[number]
            param = params[number]
            m = model(**param)
            m.fit(X_train, y_train)
            y_pred = m.predict(X_test)
            print(f'{model}')
            print(f'accureacy - {accuracy_score(y_test, y_pred):.5f}')
            print(f'precision - {precision_score(y_test, y_pred, average='weighted'):.5f}')
            print(f'recall - {recall_score(y_test, y_pred, average='weighted'):.5f}')
            print(f'roc-auc - {roc_auc_score(y_test, m.predict_proba(X_test), multi_class='ovr', average='weighted'):.5f}')
            print('')
            

In [24]:
models = [SVC, DecisionTreeClassifier, RandomForestClassifier]
params = [{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'random_state': 21, 'probability':True}, {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22, 'random_state': 21}, {'class_weight': None, 'criterion': 'gini', 'max_depth': 28, 'n_estimators': 50, 'random_state': 21}]
metrics_models(models, params)

<class 'sklearn.svm._classes.SVC'>
accureacy - 0.88757
precision - 0.89267
recall - 0.88757
roc-auc - 0.98168

<class 'sklearn.tree._classes.DecisionTreeClassifier'>
accureacy - 0.89053
precision - 0.89262
recall - 0.89053
roc-auc - 0.93806

<class 'sklearn.ensemble._forest.RandomForestClassifier'>
accureacy - 0.92899
precision - 0.93009
recall - 0.92899
roc-auc - 0.99151

